# Portfolios Example

In [1]:
%matplotlib notebook
import numpy as np
import load_factors as lf
import factors
import pickle
from factor_portfolio import Factor_Portfolio
from factor_multi_portfolio import Factor_Multi_Portfolio
from multiplicative_weigths import Multiplicative_Weights
from pattern_matching import Pattern_Matching
from algorithm_result import AlgorithmResultsList
from filter import Filter
import warnings
import time
warnings.filterwarnings('ignore')

Pre-process (Run only once)

In [ ]:
assets_list=lf.load_assets(['price_to_book'], '../data/')
prices=lf.create_factor_df(assets_list,'close',isForwardFill=False)
returns=factors.prices_to_returns(prices)
ibx_filter=Filter('../data/filter/IBX.csv')
pickle.dump(ibx_filter, open('../dump/ibx_filter.p', 'wb'))
pickle.dump(assets_list, open('../dump/assets_list.p', 'wb'))
pickle.dump(prices, open('../dump/prices.p', 'wb'))
pickle.dump(returns, open('../dump/returns.p', 'wb'))

Load objects (Run otherwise)

In [2]:
#load objects
assets_list=pickle.load(open('../dump/assets_list.p','rb'))
prices=pickle.load(open('../dump/prices.p', 'rb'))
returns=pickle.load(open('../dump/returns.p', 'rb'))
ibx_filter=pickle.load(open('../dump/ibx_filter.p', 'rb'))

Filter PN Assets only

In [3]:
#get only PN assets
assets_list=lf.get_only_PN(assets_list)

### Momentum algorithm

Create and run algorithm

In [4]:
start = time.time()
#Create
algorithm = Factor_Portfolio([], 'momentum sma', 15)
#Set parameters
algorithm.factor_df=factors.momentum(prices, 21, 252)
algorithm.reverse_order=False
algorithm.add_filter(ibx_filter)
algorithm.portfolio_size=15
algorithm.rebalance_window=1
algorithm.rebalance_period='monthly'
algorithm.cost=.004
#Run
result = algorithm.run(returns,start_date='01/01/2006')
end = time.time()
print('Factor Portfolio run time: ', (end-start), ' segundos')
print(result.summary())

Factor Portfolio run time:  2.0851588249206543  segundos
Summary:
        Yrly ROA: 0.08
        Yrly Sharpe Ratio: 0.33
        Annualized return: 8.51%
        Annualized volatility: 24.55%
        Longest drawdown: 1653 days
        Net Return: 154.42%
        Winning days: 53.2%
        Start Date: 01/2006
        End Date: 08/2017
        


In [ ]:
result.add_benchmark('../data/benchmark/IBOV.csv')
result.add_benchmark('../data/benchmark/DynamoCougar.csv')
result.plot()

In [5]:
res_list = AlgorithmResultsList()

In [6]:
res_list.append(result)

### MWU algorithm

In [ ]:
start = time.time()
algorithm = Multiplicative_Weights(eta=0.1)
algorithm.rebalance_window=1
algorithm.rebalance_period='daily'
result = algorithm.run(returns,start_date='01/01/2006')
end = time.time()
print('MWU Elapsed time: ', (end-start), ' segundos')
print(result.summary())

In [ ]:
result.add_benchmark('../data/benchmark/IBOV.csv')
result.plot()

### Pattern Matching algorithm

In [ ]:
start = time.time()
algorithm = Pattern_Matching(window_size=22,c_threshold=0.2)
algorithm.rebalance_window=1
algorithm.rebalance_period='monthly'
result = algorithm.run(returns,start_date='01/01/2006')
end = time.time()
print('Pattern Matching Elapsed time: ', (end-start), ' segundos')
print(result.summary())

In [ ]:
result.add_benchmark('../data/benchmark/IBOV.csv')
result.plot()

In [ ]:
result.data['result'].to_csv('result.csv')